# Clasificación usando scikit-learn

## Análisis diario de datos meteorológicos
Vamos a utilizar Scikit-Learn para realizar una clasificación de datos meteorológicos basada en un árbol de decisiones.

In [1]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=eb4b2f795e142e248d4942a118bc558045bfbe538a500299ea5ad8e15fc7d2ff
  Stored in directory: /home/jdg/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

### Creamos un Pandas DataFrame desde un archivo CSV

In [2]:
data = pd.read_csv('meteo/diario.csv')

## Descripción diaria de los datos meteorológicos
El archivo **meteo/diario.csv** es un archivo separado por comas que contiene datos meteorológicos. La estación meteorológica está equipada con sensores que capturan mediciones relacionadas con el clima, como la temperatura del aire, la presión del aire y la humedad relativa. Los datos se recopilaron durante un perioso de tres años, de septiembre  de 2011 a septiembre de 2014, para garantizar que se capturan datos suficientes para diferentes estaciones y condiciones climáticas.

Veamos ahora todas las columnas en los datos.

In [3]:
data.columns

Index(['number', 'air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
       'relative_humidity_3pm'],
      dtype='object')

<br>Cada fila en daily_weather.csv captura datos del tiempo para un día separado.

Las mediciones de los sensores de la estación meteorológica se capturaron a intervalos de un minuto. Estas mediciones fueron procesadas para generar valores para describir el clima diario. Dado que este conjunto de datos se creó para clasificar los días de baja humedad frente a los días de no baja humedad (es decir, días con humedad normal o alta), las variables incluidas son las mediciones del clima en la mañana, y en la tarde. La idea es usar los valores del clima de la mañana para predecir si el día tendrá poca humedad o no en función de la medición de la humedad relativa de la tarde.

Qué son cada variable?

* **number:** Número único para cada fila
* **air_pressure_9am:** presión atmosférica entre las  8:55am y las 9:04am (*Hectopascales*)
* **air_temp_9am:** temperatura promedio del aire entre las 8:55am y las 9:04am (*Grados Fahrenheit*)
* **air_wind_direction_9am:** Dirección del  8:55am to 9:04am (*Grados a donde el 0 es el Norte e incrementa en sentido antihorario*)
* **air_wind_speed_9am:** Velocidad promedio del viento de 8:55am a 9:04am (*Millas por hora*)
* ** max_wind_direction_9am:** Dirección promedio de ráfagas de 8:55am a 9:10am 
* **max_wind_speed_9am:** Velocidad promedio de las ráfagas de 8:55am a 9:04am (*Millas por hora*)
* **rain_accumulation_9am:** Cantidad de lluvia acumulada en las 24 horas previas a las 9am. (*Milímetros*)
* **rain_duration_9am:** Cantidad de tiempo en el que se registró la lluvia en las 24 horas previas a las 9 am (*segundos*)
* **relative_humidity_9am:** humedad relativa promedio de 8:55am a 9:04am (*por ciento*)
* **relative_humidity_3pm:** humedad relativa promedio de 2:55pm a 3:04pm (*por ciento*)

In [4]:
data.head(10)

,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.00,0.0,42.420000,36.160000
1,1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.00,0.0,24.328697,19.426597
2,2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.00,20.0,8.900000,14.460000
3,3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.00,0.0,12.189102,12.742547
4,4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.90,14730.0,92.410000,76.740000
5,5,915.300000,78.404000,182.800000,9.932014,189.000000,10.983375,0.02,170.0,35.130000,33.930000
6,6,915.598868,70.043304,177.875407,3.745587,186.606696,4.589632,0.00,0.0,10.657422,21.385657
7,7,918.070000,51.710000,242.400000,2.527742,271.600000,3.646212,0.00,0.0,80.470000,74.920000
8,8,920.080000,80.582000,40.700000,4.518619,63.000000,5.883152,0.00,0.0,29.580000,24.030000
9,9,915.010000,47.498000,163.100000,4.943637,195.900000,6.576604,0.00,0.0,88.600000,68.050000
